In [1]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 13.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633967 sha256=3373aed6fb71ccb208e62a2a410a595410732f3fdc6af43337c1fa72de721cc7
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
!wget "https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/movielens/movies.csv"
!wget "https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/movielens/ratings.csv"

--2022-09-02 03:17:38--  https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/movielens/movies.csv
Resolving grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)... 52.219.60.83
Connecting to grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)|52.219.60.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458390 (448K) [text/csv]
Saving to: ‘movies.csv’

movies.csv          100%[===================>] 447.65K   412KB/s    in 1.1s    

2022-09-02 03:17:40 (412 KB/s) - ‘movies.csv’ saved [458390/458390]

--2022-09-02 03:17:40--  https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/movielens/ratings.csv
Resolving grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)... 52.219.60.83
Connecting to grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)|52.219.60.83|:443... connected.
HTTP

In [3]:
!ls -tl # 리눅스 커맨드. 로컬 디렉토리에 뭐가 있는지 확인

total 2836
drwxr-xr-x 1 root root    4096 Aug 31 13:47 sample_data
-rw-r--r-- 1 root root  458390 Feb 23  2021 movies.csv
-rw-r--r-- 1 root root 2438266 Feb 23  2021 ratings.csv


# 데이터 로딩

In [4]:
from surprise import Dataset
from surprise import Reader

from collections import defaultdict
import numpy as np
import pandas as pd

In [5]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
movie_ratings = pd.merge(movies, ratings, left_on='movieId', right_on='movieId') # merge

In [9]:
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0,997938310
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0,855190091


In [10]:
movies.shape

(9125, 3)

In [11]:
ratings.shape

(100004, 4)

In [12]:
movie_ratings.shape

(100004, 6)

In [13]:
def getMovieName(movie_ratings, movieID): # 영화 title과 genres 가져옴
    return movie_ratings[movie_ratings["movieId"] == movieID][["title", "genres"]].values[0]

def getMovieID(movie_ratings, movieName): # 영화 id와 genres 가져옴
    return movie_ratings[movie_ratings["title"] == movieName][["movieId", "genres"]].values[0]

# 영화 데이터를 surprise 모듈을 통해 로딩

In [14]:
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file("ratings.csv", reader=reader) # 데이터 로딩 / 앞서 만들어 놓은 reader로 로딩을 한다. 

In [15]:
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

In [16]:
# 데이터를 훈련용과 테스트용으로 나누지 말고 모두 리턴
trainSet = data.build_full_trainset()

In [17]:
sim_options = {
    'name': 'cosine',
    'user_based': False # 아이템 기반 (user base가 False 이므로)
}

# 아이템 기반과 코사인 유사도 기반으로 동작하는 모델 생성
model = KNNBasic(sim_options=sim_options)
# 데이터로부터 아이템 벡터 생성
model.fit(trainSet) # ratings.csv 파일을 통해 훈련을 시킴
# 아이템간의 유사도 행렬 계산
simsMatrix = model.compute_similarities()

# '사용자 기반 협업 필터링-무비렌즈 데이터셋' 을 진행할 때도 이와 똑같은 과정을 거쳤음. 그때는 이 계산이 빨리 끝났음. 왜냐면 아이템의 수가 사용자의 수보다 훨씬 많았기 때문이다.
# 이 데이터셋은 사용자 수가 아이템 수보다 많아서 아이템 기반 협업 필터링 시 속도가 느림
# 그래서 이 데이터 셋은 사용자 기반 협업 필터링 시 유사도 계산 속도가 더 빠름

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [18]:
testUserId = '85' # 기준
k = 10 # 이 사용자가 좋아한 10 개의 영화를 찾고, 그 영화들과 유사도가 높은 영화들을 추출

In [19]:
pd.set_option('display.max_colwidth', -1)
display(movie_ratings[movie_ratings["userId"] == int(testUserId)].sort_values("rating", ascending=False)) # 85번 사용자가 준 평점들
# 쥬만지 좋아함
# 마리오 싫어함

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,movieId,title,genres,userId,rating,timestamp
263,2,Jumanji (1995),Adventure|Children|Fantasy,85,5.0,837511784
7443,255,"Jerky Boys, The (1995)",Comedy,85,5.0,837512635
19363,648,Mission: Impossible (1996),Action|Adventure|Mystery|Thriller,85,5.0,837511448
17405,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,85,5.0,837507044
14386,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,85,5.0,837511743
...,...,...,...,...,...,...
16429,546,Super Mario Bros. (1993),Action|Adventure|Children|Comedy|Fantasy|Sci-Fi,85,1.0,837512482
11769,366,"Wes Craven's New Nightmare (Nightmare on Elm Street Part 7: Freddy's Finale, A) (1994)",Drama|Horror|Mystery|Thriller,85,1.0,837512580
12156,374,Richie Rich (1994),Children|Comedy,85,1.0,837512723
8420,291,Poison Ivy II (1996),Drama|Thriller,85,1.0,837512611


In [20]:
testUserInnerID = trainSet.to_inner_uid(testUserId) # 내부에서 사용된 아이디 찾아야 함 

In [21]:
# 이 사용자가 좋아한 영화 k개를 읽어서 kNeighbors에 저장
testUserRatings = trainSet.ur[testUserInnerID]
kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1]) # 유사도 값을 가지고 찾은, 주어진 사용자가 좋아한 영화 10개를 대입

In [22]:
# 이 사용자 좋아한 영화들과 비슷한 영화를 찾아서 candidates에 유사도 가중치를 곱해서 저장
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = simsMatrix[itemID]
    for innerID, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating) 

In [23]:
# 사용자가 이미 본 아이템들을 기록
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

In [24]:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=lambda t: t[1], reverse=True): # 내림차순
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(movieID, getMovieName(movie_ratings, int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break # 10개만 출력하고 끝낸다
# 이 데이터셋은 사용자 기반 협업 필터링이 더 좋을 듯
# 하지만 사용자의 수가 아이템의 수보다 적은 경우는 거의 없는 편. 

3136 ['James Dean Story, The (1957)' 'Documentary'] 50.0
2607 ['Get Real (1998)' 'Drama|Romance'] 49.93620560356324
259 ['Kiss of Death (1995)' 'Crime|Drama|Thriller'] 49.83440938875971
998 ['Set It Off (1996)' 'Action|Crime'] 49.818661078285594
1935 ['How Green Was My Valley (1941)' 'Drama|Musical|Romance'] 49.71992040532635
1440 ['Amos & Andrew (1993)' 'Comedy'] 49.69868472501265
269 ['My Crazy Life (Mi vida loca) (1993)' 'Drama'] 49.691452437730206
988 ['Grace of My Heart (1996)' 'Comedy|Drama'] 49.631279483226095
2068 ['Fanny and Alexander (Fanny och Alexander) (1982)'
 'Drama|Fantasy|Mystery'] 49.628498357279526
896 ['Wild Reeds (Les roseaux sauvages) (1994)' 'Drama'] 49.58113202209386
2626 ['Edge of Seventeen (1998)' 'Comedy|Drama|Romance'] 49.565143823458385


In [25]:
def recommendForItem(userID):
    testUserInnerID = trainSet.to_inner_uid(userID)
    testUserRatings = trainSet.ur[testUserInnerID]

    kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])

    candidates = defaultdict(float)
    for itemID, rating in kNeighbors:
        similarityRow = simsMatrix[itemID]
        for innerID, score in enumerate(similarityRow):
            candidates[innerID] += score * (rating) 
 
    watched = {}
    for itemID, rating in trainSet.ur[testUserInnerID]:
        watched[itemID] = 1

    pos = 0
    for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
        if not itemID in watched:
            movieID = trainSet.to_raw_iid(itemID)
            print(movieID, getMovieName(movie_ratings, int(movieID)), ratingSum)
            pos += 1
            if (pos > 10):
                break      

In [26]:
recommendForItem('85')

3136 ['James Dean Story, The (1957)' 'Documentary'] 50.0
2607 ['Get Real (1998)' 'Drama|Romance'] 49.93620560356324
259 ['Kiss of Death (1995)' 'Crime|Drama|Thriller'] 49.83440938875971
998 ['Set It Off (1996)' 'Action|Crime'] 49.818661078285594
1935 ['How Green Was My Valley (1941)' 'Drama|Musical|Romance'] 49.71992040532635
1440 ['Amos & Andrew (1993)' 'Comedy'] 49.69868472501265
269 ['My Crazy Life (Mi vida loca) (1993)' 'Drama'] 49.691452437730206
988 ['Grace of My Heart (1996)' 'Comedy|Drama'] 49.631279483226095
2068 ['Fanny and Alexander (Fanny och Alexander) (1982)'
 'Drama|Fantasy|Mystery'] 49.628498357279526
896 ['Wild Reeds (Les roseaux sauvages) (1994)' 'Drama'] 49.58113202209386
2626 ['Edge of Seventeen (1998)' 'Comedy|Drama|Romance'] 49.565143823458385
